UBCF Notebook - Not Confidence-Aware Approach

In [303]:
import sys
sys.path.insert(0,'C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence')

In [304]:
from lenskit.algorithms import user_knn, Recommender
from lenskit.metrics.predict import rmse
from lenskit import datasets
from lenskit.datasets import MovieLens
from lenskit import crossfold as xf
from lenskit import batch, topn, datasets
from lenskit import topn, util
from lenskit.crossfold import partition_users, SampleN, partition_rows
import pandas as pd
import logging
_logger = logging.getLogger()

In [305]:
# Load MovieLens data in expected format
#ratings_df = MovieLens('../data/ml-latest-small')
ratings_df = MovieLens('../data/ml-1m')

In [4]:
util.log_to_notebook()

[   INFO] lenskit.util.log notebook logging configured


In [291]:
all_recs = []
all_preds = []
all_test = []

In [292]:
num_cv = 5
samp = xf.SampleN(5)

In [293]:
#pairs = list(partition_rows(ratings_df.ratings, num_cv))
pairs = list(partition_users(ratings_df.ratings, num_cv, samp))

[   INFO] lenskit.crossfold partitioning 1000209 rows for 6040 users into 5 partitions
[   INFO] lenskit.crossfold fold 0: selecting test ratings
[   INFO] lenskit.crossfold fold 0: partitioning training data
[   INFO] lenskit.crossfold fold 1: selecting test ratings
[   INFO] lenskit.crossfold fold 1: partitioning training data
[   INFO] lenskit.crossfold fold 2: selecting test ratings
[   INFO] lenskit.crossfold fold 2: partitioning training data
[   INFO] lenskit.crossfold fold 3: selecting test ratings
[   INFO] lenskit.crossfold fold 3: partitioning training data
[   INFO] lenskit.crossfold fold 4: selecting test ratings
[   INFO] lenskit.crossfold fold 4: partitioning training data


In [294]:
#agg_type = 'average'
agg_type = 'weighted-average'
#agg_type = 'sum'

In [295]:
nbhd_sizes = [10, 25, 50, 100, 250] #[10, 25, 50, 100, 150, 250]

In [296]:
cv_fold = 0

for train, test in pairs: #: # # 
    cv_fold += 1
    _logger.info('cv fold: %d', cv_fold)
    test['RunId'] = cv_fold #train['RunId'] = cv_fold
    _logger.info('Training set size: %d | Test set size: %d', len(train), len(test))
    all_test.append(test)

    for f in nbhd_sizes:
        _logger.info('Starting neighborhood size: %d', f)
        
        _logger.info('Configuring algorithm...')
        # configure the algorithm
        ####algo = user_knn.UserUser(nnbrs = f, aggregate = agg_type) # weighted-average # average
        algo = user_knn.UserUser(nnbrs = f, aggregate = agg_type) # Confidence-Aware UBCF
        ####algo = als.BiasedMF(50)
        # make sure it can recommend, not just predict
        algo = Recommender.adapt(algo)

        # train the algorithm
        algo.fit(train) # [:-1000]

        ###algo.fit(train)
        print('Starting prediction process...')
        # predict ratings
        preds = batch.predict(algo, test)
        preds['nnbhrs'] = f
        preds['RunId'] = cv_fold
        all_preds.append(preds)
        print('Starting recommendation process...')
        # produce recommendations
        users = test['user'].unique()
        recs = batch.recommend(algo, users, 50) # 20
        recs['RunId'] = cv_fold
        recs['nnbhrs'] = f
        all_recs.append(recs)

        _logger.info('Finishing neighborhood size: %d', f)

[   INFO] root cv fold: 1
[   INFO] root Training set size: 994169 | Test set size: 6040
[   INFO] root Starting neighborhood size: 10
[   INFO] root Configuring algorithm...
[   INFO] lenskit.algorithms.basic trained unrated candidate selector for 994169 ratings
Starting prediction process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=10, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104144 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 6040 predictions for 1208 users (setup took  86ms)
[   INFO] lenskit.batch._predict generated 6040 predictions for 1208 users in 20.38s
Starting recommendation process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=10, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104144 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._recommend recommending with TopN/UserUser(nnbrs=10,

[   INFO] lenskit.batch._predict generated 6040 predictions for 1208 users in 20.08s
Starting recommendation process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=50, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104124 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._recommend recommending with TopN/UserUser(nnbrs=50, min_sim=0) for 1208 users (n_jobs=None)
[   INFO] lenskit.batch._recommend recommended for 1208 users in 44.02s
[   INFO] root Finishing neighborhood size: 50
[   INFO] root Starting neighborhood size: 100
[   INFO] root Configuring algorithm...
[   INFO] lenskit.algorithms.basic trained unrated candidate selector for 994169 ratings
Starting prediction process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=100, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104124 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batc

[   INFO] root Training set size: 994169 | Test set size: 6040
[   INFO] root Starting neighborhood size: 10
[   INFO] root Configuring algorithm...
[   INFO] lenskit.algorithms.basic trained unrated candidate selector for 994169 ratings
Starting prediction process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=10, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104124 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 6040 predictions for 1208 users (setup took  53ms)
[   INFO] lenskit.batch._predict generated 6040 predictions for 1208 users in 19.31s
Starting recommendation process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=10, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104124 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._recommend recommending with TopN/UserUser(nnbrs=10, min_sim=0) for 1208 users

Starting recommendation process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=50, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104124 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._recommend recommending with TopN/UserUser(nnbrs=50, min_sim=0) for 1208 users (n_jobs=None)
[   INFO] lenskit.batch._recommend recommended for 1208 users in 44.38s
[   INFO] root Finishing neighborhood size: 50
[   INFO] root Starting neighborhood size: 100
[   INFO] root Configuring algorithm...
[   INFO] lenskit.algorithms.basic trained unrated candidate selector for 994169 ratings
Starting prediction process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=100, min_sim=0) to 1121 pickle bytes with 13 buffers of 28104124 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 6040 predictions for 1208 users (setup took  56ms)
[   INFO] le

In [297]:
recs = pd.concat(all_recs, ignore_index=True)
preds = pd.concat(all_preds, ignore_index=True)
test = pd.concat(all_test, ignore_index=True)

In [209]:
preds
#train
#test

,user,item,rating,timestamp,RunId,prediction,nnbhrs
0,1,1287,2.0,1260759187,1,2.727396,10
1,1,1371,2.5,1260759135,1,1.809818,10
2,1,3671,3.0,1260759117,1,2.712714,10
3,1,1029,3.0,1260759179,1,2.140819,10
4,1,31,2.5,1260759144,1,2.297384,10
...,...,...,...,...,...,...,...
3350,670,25,5.0,938782344,5,4.122230,10
3351,670,47,5.0,938782050,5,4.240131,10
3352,670,457,4.0,938782414,5,4.048139,10
3353,670,1183,5.0,938782193,5,4.152512,10


In [ ]:
preds.loc[preds['prediction'] > 5,'prediction'] = 5
preds.loc[preds['prediction'] < 1,'prediction'] = 1

#dfd.loc[mask, 'c'] = 42

In [210]:
preds['prediction'].describe()

count    3301.000000
mean        3.721255
std         0.755124
min         0.244807
25%         3.240762
50%         3.773771
75%         4.247910
max         5.903743
Name: prediction, dtype: float64

In [83]:
preds[['nnbhrs','prediction']].groupby(['nnbhrs']).describe()

prediction                                                              \
            count      mean       std       min       25%       50%       75%   
nnbhrs                                                                          
10        30190.0  3.798911  0.754895 -0.527385  3.357596  3.880937  4.334508   
25        30186.0  3.807101  0.727728 -0.394192  3.383944  3.887675  4.321402   
50        30180.0  3.813000  0.719385 -0.394192  3.390688  3.894344  4.324512   
100       30146.0  3.816649  0.715424 -0.394192  3.397159  3.898184  4.325361   
150       30112.0  3.817199  0.713379 -0.394192  3.396018  3.897624  4.326785   
250       30058.0  3.814306  0.710722 -0.394192  3.392338  3.891877  4.322018   

                  
             max  
nnbhrs            
10      6.032298  
25      5.927638  
50      5.848282  
100     5.818248  
150     5.802726  
250     5.756113

In [84]:
# compute RMSE
print('RMSE:', rmse(preds['prediction'], preds['rating'])) 

# ML100k - partition users - average - RMSE = 0.9157690305188498, 0.91377232782083
# ML100k - partition users - weighted-average - RMSE = 0.9185319353148285, 0.9333875278175761
# ML100k - partition users - sum - RMSE = 1.3963497485089527


RMSE: 0.9247950824213405


In [298]:
#preds.groupby(['nnbhrs'])['prediction', 'rating'].rmse()
preds['err'] = (preds['prediction'] - preds['rating'])**2

In [299]:
import numpy as np
np.sqrt(preds[['err', 'nnbhrs']].groupby(['nnbhrs']).mean() )#.groupby(['nnbhrs']).sqrt() #sqrt()
#preds.groupby(['nnbhrs']).sqrt()

,err
nnbhrs,
10,0.937873
25,0.920817
50,0.919118
100,0.921558
250,0.927426


In [213]:
recs # single nbhr size

,item,score,user,rank,RunId,nnbhrs
0,91104,5.783333,1,1,1,10
1,106441,5.165356,1,2,1,10
2,107559,5.165356,1,3,1,10
3,3216,4.920168,1,4,1,10
4,40412,4.920168,1,5,1,10
...,...,...,...,...,...,...
13415,27478,6.025225,670,16,5,10
13416,3892,6.004789,670,17,5,10
13417,2267,5.811897,670,18,5,10
13418,3437,5.806824,670,19,5,10


In [214]:
test # single nbhr size

,user,item,rating,timestamp,RunId
0,1,1287,2.0,1260759187,1
1,1,1371,2.5,1260759135,1
2,1,3671,3.0,1260759117,1
3,1,1029,3.0,1260759179,1
4,1,31,2.5,1260759144,1
...,...,...,...,...,...
3350,670,25,5.0,938782344,5
3351,670,47,5.0,938782050,5
3352,670,457,4.0,938782414,5
3353,670,1183,5.0,938782193,5


In [215]:
# compute nDCG for each user
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
rla.add_metric(topn.precision)
metrics = rla.compute(recs, test)
# and compute overall system performance
print('NDCG:', metrics['ndcg'].mean())
print('Precision:', metrics['precision'].mean())

[   INFO] lenskit.topn analyzing 13420 recommendations (3355 truth rows)
[   INFO] lenskit.topn using rec key columns ['nnbhrs', 'user', 'RunId']
[   INFO] lenskit.topn using truth key columns ['user', 'RunId']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.util.parallel setting up in-process worker
[   INFO] lenskit.topn measured 671 lists in 6.69s
NDCG: 0.0
Precision: 0.0


In [219]:
print('NDCG:', metrics['ndcg'].mean())
print('Precision:', metrics['precision'].mean())

TypeError: list indices must be integers or slices, not str

In [300]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
rla.add_metric(topn.precision)
metrics = pd.DataFrame() 

for f in nbhd_sizes:
    _logger.info('Starting neighborhood size: %d', f)
    recs_f = recs[(recs['nnbhrs'] == f)]
    del recs_f['nnbhrs']
    metrics_f = rla.compute(recs_f, test)
    metrics_f['nbhr_size'] = f
    metrics_f = pd.DataFrame(metrics_f)
    metrics = pd.concat([metrics, metrics_f])

[   INFO] root Starting neighborhood size: 10
[   INFO] lenskit.topn analyzing 301950 recommendations (30200 truth rows)
[   INFO] lenskit.topn using rec key columns ['user', 'RunId']
[   INFO] lenskit.topn using truth key columns ['user', 'RunId']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm serialized <lenskit.topn._RLAJob object at 0x000002111DC47B80> to 2646854 pickle bytes with 24164 buffers of 12923600 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.topn measured 6039 lists in 1m4.76s
[   INFO] root Starting neighborhood size: 25
[   INFO] lenskit.topn analyzing 301950 recommendations (30200 truth rows)
[   INFO] lenskit.topn using rec key columns ['user', 'RunId']
[   INFO] lenskit.topn using truth key columns ['user', 'RunId']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm 

In [175]:
# DELETE
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
rla.add_metric(topn.precision)

In [183]:
# DELETE
f = 10
recs_f = recs[(recs['nnbhrs'] == f)]
del recs_f['nnbhrs']
recs_f

,item,score,user,rank,RunId
0,91104,7.092982,5,1,1
1,106441,6.558338,5,2,1
2,107559,6.558338,5,3,1
3,3879,6.172217,5,4,1
4,55250,6.092982,5,5,1
...,...,...,...,...,...
13415,32289,5.198280,665,16,5
13416,41573,5.198280,665,17,5
13417,48791,5.198280,665,18,5
13418,80839,5.198280,665,19,5


In [110]:
# DELETE
del test['Unnamed: 0']

In [264]:
#pd.DataFrame(metrics_f) #= rla.compute(recs_f, test)
metrics

,,nrecs,ndcg,precision,nbhr_size
user,RunId,,,,
1,1,20.0,0.0,0.0,10
2,3,20.0,0.0,0.0,10
3,4,20.0,0.0,0.0,10
4,1,20.0,0.0,0.0,10
5,4,20.0,0.0,0.0,10
...,...,...,...,...,...
667,2,20.0,0.0,0.0,25
668,3,20.0,0.0,0.0,25
669,1,20.0,0.0,0.0,25


In [301]:
metrics[['nbhr_size','precision']].groupby(['nbhr_size']).mean()

,precision
nbhr_size,
10,0.005170
25,0.005561
50,0.005809
100,0.005922
250,0.006008


In [302]:
metrics[['nbhr_size','ndcg']].groupby(['nbhr_size']).mean()

,ndcg
nbhr_size,
10,0.020077
25,0.020556
50,0.020585
100,0.020276
250,0.019753
